# Topic modelling
This notebook explores the science of topic modelling, where from a text dataset we want to obtain similar topics. [There are many authors that explored this](http://delivery.acm.org/10.1145/2140000/2133826/p77-blei.pdf?ip=77.227.30.78&id=2133826&acc=OPEN&key=4D4702B0C3E38B35%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35%2E6D218144511F3437&__acm__=1568126591_4585170a693dc7e37a93b845996026ca).

This is going to be a practical hands-on experience notebook where we will cover the following:

 - Text preprocessing.
 - Text transformation (TF-IDF, Word2vec)
 - Algorithms based on signal decomposition:
     - Latent Dirichlet Allocation. (Using TF-IDF).
     - Non-negative Matrix Factorization (using TF-IDF).
 - Clustering algorithms:
     - K-means + TF-IDF.
 - Algorithms based on similarity:
     - WordVectors + Cosine similarity + np masked array.

Please note that **all the models require a text transformation**. That's it, either using TD-IDF or Word2Vec.

In [1]:
from IPython.core.display import Image, display

from IPython.display import HTML
HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    To toggle <a href="javascript:code_toggle()">on/off</a> the raw code.''')



In [59]:
#libraries
import pandas as pd
import numpy as np
from time import time
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics

In [3]:
from sklearn.datasets import fetch_20newsgroups

In [4]:
# set model parameters:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

## Sample data downloading
In this notebook we will be using the 20 newsgroup dataset. 
Scikit has an utility that can download datasets.

In [6]:
print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 180.182s.


For the sake of simplicity we will create a Pandas Series, so all functions here ara compatible with a dataframe

In [70]:
texts = pd.Series(data_samples)

In [71]:
display(texts.head(5))

0    Well i'm not sure about the story nad it did s...
1    \n\n\n\n\n\n\nYeah, do you expect people to re...
2    Although I realize that principle is not one o...
3    Notwithstanding all the legitimate fuss about ...
4    Well, I will have to change the scoring on my ...
dtype: object

We will use some utility functions that print the top words per topic and to produce a dataframe where we will compare the assigned topic for each different technique.

In [44]:
#utils function
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [45]:
def cluster_topics(model,tf):
    topic_matrix = lda.transform(tf)
    topics = []
    for ele in topic_matrix:
        topics.append(np.argmax(ele))
    return topics

## Text cleaning
We are going to remove:
- non-printable characters.
- punctuations: please note that this would remove some meaning and it won't be successful with word vectors.
- standalone numbers and standalone characters.

In [72]:
tr = str.maketrans(string.punctuation,' '*len(string.punctuation))
single_char = r"\b[a-zA-Z]\b"
number_regex = r"\b\d+\b"

In [73]:
#remove punctuation
texts = texts.str.translate(tr)
#remove single chars
texts = texts.str.replace(single_char,'')
#remove all numbers
texts = texts.str.replace(number_regex,'')
#remove line breaks win,linux, macOS
texts = texts.str.replace('\n','')
texts = texts.str.replace('\r','')
#remove tabs:
texts = texts.str.replace('\t','')
# remove trailing whitespaces
texts = texts.str.strip()
# remove whitespaces in string
texts = texts.str.replace(' +', ' ')

In [74]:
display(texts.head(5))

0    Well not sure about the story nad it did seem ...
1    Yeah do you expect people to read the FAQ etc ...
2    Although realize that principle is not one of ...
3    Notwithstanding all the legitimate fuss about ...
4    Well will have to change the scoring on my pla...
dtype: object

## Text preprocessing
We are going to:
 - lowercase the text.
 - tokenize.
 - vectorize and calculate the TF-IDF.
Some other preprocessing not done here [check this link from Stanford NLP group](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html). These techniques are useful when using Bag of Words approaches.:
 - Stemming: getting the root of the word by applying heuristics (see Porter's Stemmer).
 ![Sample of Stemm](img/img100.png)
 - Lemmatization: getting the root of the words by using morphological analysis.
 


In [75]:
texts = texts.str.lower()

In [76]:
display(texts.head(5))

0    well not sure about the story nad it did seem ...
1    yeah do you expect people to read the faq etc ...
2    although realize that principle is not one of ...
3    notwithstanding all the legitimate fuss about ...
4    well will have to change the scoring on my pla...
dtype: object

## Text transformation
Remember the algorithms only understand a numerical representation, so we have to choose:
 - Term frecuency. (TF)
 - Term frecuency, inverse document frecuency. (TF-IDF).
 - Word Vectors.
 
Please note that there are other approaches like bags of words. See the tutorial [working with text data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) from Scikit-learn.

### Create a full tf-idf vectorizer for our text:

In [107]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_components,
                                   stop_words='english')

In [108]:
tfidf = tfidf_vectorizer.fit_transform(texts)

### Create a term frecuency vectorizer:

In [109]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_components,
                                stop_words='english')

In [110]:
tf = tf_vectorizer.fit_transform(texts)

# Latent Dirichlet Allocation:
The graphical model of LDA is a three-level generative model:
See the [scikit link.](https://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation)

![Latent Diriletch Allocation](img/lda_model_graph.png)


Note on notations presented in the graphical model above, which can be found in Hoffman et al. (2013):

 - The corpus is a collection of *D* documents.
 - A document is a sequence of *W* words.
 - There are *K* topics in the corpus.
 
The boxes represent repeated sampling.

 - Method explained here: http://jmlr.csail.mit.edu/papers/v3/blei03a.html
 - Classification example: https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

We are going to:
 - Select the language (in this case english).
 - Preprocess the text (**In this example we will consider the text is already preprocessed**)
 - Compute the tf-idf for the text.
 - Compute the LDA
 - Use the Non-negative factor matrix decomposition to get the topics

In [111]:
from sklearn.decomposition import LatentDirichletAllocation

## Fit LDA

In [112]:
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [113]:
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

done in 1.721s.


In [114]:
#topics(lda,texts)
cluster_lda = pd.Series(cluster_topics(lda,tf),index=texts.index)

In [115]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: just like know don people time new think use edu
Topic #1: know don like people think just time use new edu
Topic #2: don think use like just people new know time edu
Topic #3: think time just don like people know use new edu
Topic #4: people time know new like don just use think edu
Topic #5: just think like edu know use new don time people
Topic #6: new time just know use don like think edu people
Topic #7: edu like use time new know don just people think
Topic #8: use know don just people like time think edu new
Topic #9: like time use new just think don know people edu



# Non-negative matrix factorization
Two methods:
 - Frobenius norm
 - Generalized Kullback-Leibler divergence

In [116]:
from sklearn.decomposition import NMF

In [117]:
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

done in 0.346s.


In [118]:
cluster_nmf = pd.Series(cluster_topics(nmf,tfidf),index=texts.index)

In [119]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

Topic #0: people just think know time don like use new edu
Topic #1: edu use time think people new like know just don
Topic #2: like just don know use time think people new edu
Topic #3: time like new just people know think don use edu
Topic #4: know don people like just time use think new edu
Topic #5: just like don think people time know new use edu
Topic #6: think don just people like time use new know edu
Topic #7: new time just like use think people know edu don
Topic #8: use like just don time think people new know edu
Topic #9: don know just like think people use time new edu



In [120]:
t0 = time()
nmf2 = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

done in 0.158s.


In [121]:
cluster_nmf2 = pd.Series(cluster_topics(nmf2,tfidf),index=texts.index)

In [122]:
# tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf2, tfidf_feature_names, n_top_words)

Topic #0: people think know don like just use time new edu
Topic #1: edu use time think people new like know just don
Topic #2: like think just know don use time people new edu
Topic #3: time know like people just use think new edu don
Topic #4: know think don just use time people new like edu
Topic #5: just think know like don use time people new edu
Topic #6: time think use people new like know just edu don
Topic #7: new think use time people like know just edu don
Topic #8: use know like time think people new just edu don
Topic #9: don know think like just use time people new edu



# Clustering with K-means
You can check the full example [here](https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py).

In this case we will not reduce the dimensionality (using LSA), neither normalizing the input (using a truncated SVD + Normalized).

**The number of clusters is not known beforehand**. The algorithm needs as input the number of cluster. A **good** estimation is to use the Silhouette coefficient (a value in teh range [0,1]). The main drawback is that in order to do the estimation, we have to compute the k-means and then the silhouette coefficient.

In this example we will compute manually for 2 and 10 clusters, then we will compute the silhouette.

In [123]:
km = KMeans(n_clusters=2)

In [124]:
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(tfidf)
print("done in %0.3fs" % (time() - t0))

Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
done in 0.934s


In [125]:
s = metrics.silhouette_score(tfidf, km.labels_)
print("Silhouette score for 2 clusters: %s" % s)

Silhouette score for 2 clusters: 0.22292668296056226


Now, let's try with 10 clusters:

In [126]:
km = KMeans(n_clusters=10)

In [127]:
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(tfidf)
print("done in %0.3fs" % (time() - t0))

Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
done in 1.347s


In [128]:
s = metrics.silhouette_score(tfidf, km.labels_)
print("Silhouette score for 10 clusters: %s" % s)

Silhouette score for 10 clusters: 0.5173402000768418


In [129]:
#now let's get the cluster number per text:
cluster_kmeans = pd.Series(km.labels_,index=texts.index)

# Clustering using the cosine similarity
This technique uses word vectors approach. The text is transformed into a representation by wordvectors. Then, in a vectorized approach, using numpy.

In [79]:
import spacy

In [81]:
nlp = spacy.load('en_core_web_lg')

In [84]:
docs = [nlp(i) for i in data_samples]

In [85]:
simma = np.ma.arange(len(docs))

In [86]:
comparer = lambda x: docs[idx].similarity(docs[x])

In [87]:
vfunc = np.vectorize(comparer)

In [89]:
similar_topics = []
thr = 0.89
t0 = time()
for idx,val in enumerate(simma):
    if simma[idx] is not np.ma.masked:
        res = vfunc(simma)
        topic_sim = np.ma.masked_where(res <= thr,res)
        similar_topics.append(simma[~topic_sim.mask].compressed())
        simma[~topic_sim.mask] = np.ma.masked
print("done in %0.3fs" % (time() - t0))

done in 4.668s


In [98]:
mapping = np.zeros(len(docs))
for idx,i in enumerate(similar_topics):
    for elem in i:
        mapping[elem] = idx

In [100]:
cluster_cosine =  pd.Series(mapping,index=texts.index)

# Comparison

In [130]:
clusters_per_documents = pd.DataFrame({'lda':cluster_lda,'nmf1':cluster_nmf,'nmf2':cluster_nmf2,'kmeans':cluster_kmeans,'cosine':cluster_cosine})

In [133]:
topics = {}
for i in range(0,10):
    topics[i] = (clusters_per_documents == i)

In [141]:
clusters_per_documents[clusters_per_documents == 0].count().index

Index(['lda', 'nmf1', 'nmf2', 'kmeans', 'cosine'], dtype='object')

In [142]:
clusters_per_documents.corr()

,lda,nmf1,nmf2,kmeans,cosine
lda,1.000000,0.943126,0.943126,0.232121,-0.153717
nmf1,0.943126,1.000000,1.000000,0.233899,-0.161572
nmf2,0.943126,1.000000,1.000000,0.233899,-0.161572
kmeans,0.232121,0.233899,0.233899,1.000000,-0.218054
cosine,-0.153717,-0.161572,-0.161572,-0.218054,1.000000


In [159]:
clusters_per_documents[['cosine','lda','kmeans']].groupby(['lda','kmeans']).agg({'cosine':['count']})

cosine
            count
lda kmeans       
0   0         646
    2          11
    4           2
    5         116
    6           3
    7           2
    8          18
    9           9
1   1          18
    2           3
    3           1
    4           8
    5          10
    6           9
    7         134
    8          12
    9          28
2   1           1
    8          97
3   1         118
    2           2
    3           3
    4           1
    5           1
    6          10
    8           3
    9          16
4   0          51
    2           4
    4           3
    5           1
    6          11
    7           5
    8           4
    9         124
6   2           1
    4           1
    6         141
7   4         105
8   1           2
    2         114
    4           2
    6           1
9   2          16
    3         119
    5           1
    6          10
    8           2

In [156]:
clusters_per_documents[['lda','cosine']].groupby('cosine').count().nlargest(20,'lda')

,lda
cosine,
0.0,1562
2.0,59
19.0,35
3.0,31
1.0,20
7.0,18
29.0,14
34.0,12
16.0,10
